Calculations in OMUSE are done with quantities having units - the interfaces to simulation codes require it even. These quantities and their units are implemented as python classes and can be used almost everywere you would normaly use a number (or a numpy array). 

The units can be imported from the omuse.units module:

In [11]:
from omuse.units import units,constants

By itself a unit is not so useful, they conform to a unit algebra which can be used to construct new units:

In [43]:
m2=units.m*units.m
cm=units.m/100
print m2
print cm

m * m
0.01 * m


You can create a *quantity* by typing a number and combining it with a unit using the bar `|` operator.

In [9]:
print 10. | units.kg
print 5 | m2

10.0 kg
5.0 m**2


Most operations you can do on numbers, you can also perform on quantities. For example, assuming the earth is a sphere, let's calculate some properties:

In [12]:
earth_radius = 6384 | units.km
print "diameter :", earth_radius * 2
print "surface area :", 4.0 * constants.pi * earth_radius**2
print "volume :", 4.0 / 3.0 * constants.pi * earth_radius**3

diameter : 12768000.0 m
surface area : 5.12148164653e+14 m**2
volume : 1.08985129438e+21 m**3


It is also possible to combine quantities with different units in a calculation. To continue our properties of the earth example, lets calcute it's density:

In [13]:
earth_mass = 5.97219e24 | units.kg
earth_volume = 4.0 / 3.0 * constants.pi * earth_radius**3

print "density :", earth_mass / earth_volume

density : 5479.82099098 m**-3 * kg


You will want to interact with other python libraries that simply cannot handle units. For those cases you can convert the quantity into a value of a specific unit:

In [5]:
print "earth mass in gram :", earth_mass.value_in(units.g)

earth mass in gram : 5.97219e+27


Conversion to a different unit is also supported:

In [16]:
print "earth mass:", earth_mass.in_(units.g)

earth mass: 5.97219e+27 g


Numpy arrays and python lists can also be converted to quantities. Once converted, the resulting vector quantities support a lot of numpy operations

In [20]:
masses = [1.,2.,3.] | units.kg
print "sum of masses: ", masses.sum()
import numpy
numpy.std(masses)

sum of masses:  6.0 kg


quantity<0.816496580928 kg>

You can create your own unit with a new name using the `units.named` function. This functions takes the name of the unit, a symbol for printing and the unit it is based on. You can define a unit to represent the volume of the earth

In [21]:
earth_volume_unit = units.named('Vol-Earth', 'EarthVol', earth_volume.to_unit())
print earth_volume.as_quantity_in(earth_volume_unit)

1.0 EarthVol


Most operations on a vector quantity are elementwise. For example (As OMUSE uses numpy internally for these operations we refer to the numpy documentation, if you want to learn more):

In [25]:
sides = [ 10. , 10. , 20.] | units.cm
volumes = radii**3
print "densities :", (masses / volumes)

densities : [1000.0, 2000.0, 375.0] m**-3 * kg


Quantities become normal numbers or numpy arrays when the units cancel out in a calcution (You can use this fact, to replace a `value_in` function call with a division):

In [26]:
print volumes / earth_volume

[  9.17556372e-25   9.17556372e-25   7.34045098e-24]


Operations with incompatible units will fail:

In [27]:
print earth_mass + earth_volume

IncompatibleUnitsException: Cannot express 1000000000.0 * m**3 in kg, the units do not have the same bases

In addition to using the normal (S.I.) based unit system, there are cases where it is useful to be able to communicate 
with codes that internally use scaleless or under specified units without specifying a definite unit system, but 
with the advantages of using units (checking for consistency). Although the quantities involved do not have a specific unit base, they still have a dimension (mass, length etc.). In AMUSE we can use *generic units* in this case. In other words, you can specify if a value has a *mass*, *length* or *time* dimension, or any combination thereof, such as *length* per *time* etc.

In [30]:
from amuse.units import generic_unit_system, generic_unit_converter

Quantities with generic units work exactly the same as quantities with normal (**S.I.**) units.

In [36]:
m = 1.0 | generic_unit_system.mass
v = 0.1 | generic_unit_system.length / generic_unit_system.time
print 0.5*m*v**2
print (0.5*m*v**2).value_in(generic_unit_system.energy)

0.005 length**2 * time**-2 * mass
0.005


A unit converter can be used to convert between the systems of units, the following defines a converter between a unit system with G=1, an Astronomical Unit = 1, and the solar mass = 1:

In [40]:
converter = generic_unit_converter.ConvertBetweenGenericAndSiUnits(1 | units.MSun, 1 | units.AU, constants.G)
print "Mass of the sun, scaled:" , converter.to_generic(1 | units.MSun)
print "10 generic masses, in S.I.:" , converter.to_si(10 | generic_unit_system.mass)
print "1 generic time, in S.I:" , converter.to_si(1 | generic_unit_system.time).as_quantity_in(units.yr)
print "10 km/s, in generic:" , converter.to_generic(10.0 | units.km / units.s)

Mass of the sun, scaled: 1.0 mass
10 generic masses, in S.I.: 1.98892e+31 kg
1 generic time, in S.I: 0.159141174195 yr
10 km/s, in generic: 0.335700383529 length * time**-1


Converters can be defined with any combination of constraints, as long as they are not degenerate and span the 
unit space required.

In [41]:
natural_units_convert = generic_unit_converter.ConvertBetweenGenericAndSiUnits(constants.c,
                                  constants.G,
                                  constants.hbar,
                                  1/(4*numpy.pi*constants.eps0),
                                  constants.kB)

M = 1 | generic_unit_system.mass
T = 1 | generic_unit_system.time
L = 1 | generic_unit_system.length
Q = 1 | generic_unit_system.charge
THETA= 1| generic_unit_system.temperature

print natural_units_convert.to_si(M).in_(units.kg)
print natural_units_convert.to_si(T).in_(units.s)
print natural_units_convert.to_si(L).in_(units.m)
print natural_units_convert.to_si(Q).in_(units.C)
print natural_units_convert.to_si(THETA).in_(units.K)

2.17643740821e-08 kg
5.39123788657e-44 s
1.61625245768e-35 m
1.87554586987e-18 C
1.41678472104e+32 K
